In [1]:
pID = 1
pID = 'sub-0' + "%02d" % (pID)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score, KFold
import pickle
import scipy.io
from bci_funcs import windowed_mean, base_correct, drop_baseline

path = '/Users/lukasgehrke/Documents/publications/2021-fastReach/data/study/eeglab2python/'

In [3]:
pre_move = scipy.io.loadmat(path+pID+'/pre_move.mat')
idle = scipy.io.loadmat(path+pID+'/idle.mat')
chans = np.array(pd.read_csv(path+pID+'/sel_chans.csv')).flatten()
chans = chans - 1 # Matlab to python indexing

filename = path+pID+'/chans_'+pID+'_eeg.sav'
pickle.dump(chans, open(filename, 'wb'))

windows = 11
baseline_ix = 1

idle = idle['idle'][chans,:,:]
pre_move = pre_move['pre_move'][chans,:,:]

idle_means = np.zeros((idle.shape[2], idle.shape[0] * (windows-baseline_ix)))
pre_move_means = np.zeros((idle_means.shape))

for trial_ix in range(0, idle.shape[2]-1):
    tmp = base_correct(windowed_mean(pre_move[:,:,trial_ix], windows = windows))
    pre_move_means[trial_ix, :] = drop_baseline(tmp, baseline_ix).flatten()
    tmp = base_correct(windowed_mean(idle[:,:,trial_ix], windows = windows))
    idle_means[trial_ix, :] = drop_baseline(tmp, baseline_ix).flatten()

data = np.concatenate((pre_move_means, idle_means), axis = 0)

In [4]:
pre_move_class = np.ones((pre_move_means.shape[0], 1))
idle_class = np.zeros((idle_means.shape[0], 1))
classes = np.concatenate((pre_move_class, idle_class)).ravel()

In [5]:
clf = LDA(solver='lsqr', shrinkage='auto')
clf.fit(data, classes)
print(clf.coef_.shape)

kfolds = KFold(n_splits=10, random_state=1, shuffle=True) 
cv_results = cross_val_score(clf, data, classes, cv=kfolds)
print(cv_results.mean())
print(cv_results.std())

filename = path+pID+'/model_'+pID+'_eeg.sav'
pickle.dump(clf, open(filename, 'wb'))

(1, 190)
0.26666666666666666
0.2260776661041756
